# Mixtures of Factor Analyzers
## Multivariate Normal Distribution
### Joint Distribution
### Conditional Distribution is Normal when Jointly Distributed RVs have a Normal distributed 
### Marginal Distribution is Normal when Jointly Distributed RVs have a Normal distributed 
## Factor Analysis Model
### Assumption about Continuous Latent Variable 
### Incomplete Log-Likelihood
### Jensen's Inequality and the Complete Log-Likelihood
## Mixtures of Factor Analyzers
### Assumption about the discrete and continuous latent Variable
### Incomplete Log-Likelihood
### Complete Log-Likelihood and Jensen's Inequality
### Alteranting Expectation Conditional Maximization Algorithm (AECM)
#### First E-step for discrete latent variable
#### Maximization w.r.t Mixing paramter and mean and covariance matrix of the distribution in each cluster
#### Second E-step for continuous latent variable
#### Maximization w.r.t parameters of the marginal distribution 
#### Convergence Test
#### Initialization Options
### Predicting to which cluster an observation belongs to

In [1]:
%matplotlib inline
import numpy as np 
import sklearn.preprocessing
import sklearn.datasets
import pandas as pd
import sklearn.model_selection
import numpy.random
import math
import sklearn.metrics
import scipy.stats
import scipy.special
import matplotlib.pyplot as plt


In [94]:
#To have better initialization of the algorithm
class kmeans(object):

    def __init__(self, X_train, k):
        self.K = k
        self.m = X_train.shape[0]
        self.n = X_train.shape[1]
        self.X_train = X_train
        choices = numpy.random.choice(np.arange(0, self.m), self.K, replace=False)
        self.centers = [X_train[choices[i], :].reshape(-1, 1) for i in range(0, self.K)]# initalize the clusters centers to be one of the observations
        self.clusters_assignments = np.zeros((self.m, 1))#Just to give it the necessary shape
    
    def reassign_clusters_centers(self):
        for k in range(0, self.K):
            temp = np.zeros((self.n, 1))
            clusters = list(map(lambda i: True if i == k else False, self.clusters_assignments))
            for i in range(0, self.m):
                if clusters[i] == True:
                    temp += clusters[i] * self.X_train[i, :].reshape(-1, 1)#clusters contained in {0, 1}
                else:
                    pass
            #print(np.sum(clusters))
            self.centers[k] = temp/np.sum(clusters)

    def distortiuon_function(self):
        temp = 0
        for i in range(0, self.m):
            for k in range(0, self.K):
                if self.clusters_assignments[i] == k:
                    temp += np.linalg.norm(self.X_train[i, :].reshape(-1, 1) - self.centers[k].reshape(-1, 1))**2
                    break#They willn't be assigned to more than one cluster in tandem
        return temp

    def assign_to_clusters(self, x):
        temp = []
        for k in range(0, self.K):
            temp.append(np.linalg.norm(x.reshape(-1, 1) - self.centers[k].reshape(-1, 1))**2)#We will use L2-norm for dissimilarity measure
        return np.argmin(temp)#return the cluster number

    def E_step(self):
        for i in range(0, self.m):
            self.clusters_assignments[i] = self.assign_to_clusters(X_train[i, :].reshape(-1, 1))
    
    def fit(self, max_iterations, eps=1e-5):
        self.E_step()#To initialize the clusters assignments
        past = 10
        future = 0
        count = 0
        while(abs(past - future) > eps):#I will care for only lack of progress because k-means will always be able to minimize the distortion functions
            print(f"count:{count}, max_iterations{max_iterations}, past:{past}, future:{future}")
            count += 1
            past = self.distortiuon_function()
            self.reassign_clusters_centers()#The M step
            self.E_step()
            future = self.distortiuon_function()

        return self.centers, self.clusters_assignments

    def prediction_dataset(self, X):
        predictions = []
        for i in range(0, X.shape[0]):
            predictions.append(self.assign_to_clusters(X[i, :].reshape(-1, 1)))
        return predictions

    def predict(self, x):
        return  self.assign_to_clusters(x.reshape(-1, 1))



In [178]:
class Mixtures_Factor_Analyzers_Model(object):

    def __init__(self, X_train, G, sizeOfFactorLoadingMatrix, initialization_type="random", max_iterations=100):
        if G != len(sizeOfFactorLoadingMatrix):
            raise Exception("The number of Clusters doesn't match the number of factor loading matrix")
        
        self.m = X_train.shape[0]
        self.n = X_train.shape[1]
        self.X_train = X_train
        self.G = G#number of clusters
        self.loadingSize = sizeOfFactorLoadingMatrix
        #Just Temporary values
        self.mixing_parameter = np.zeros((self.G, 1))
        self.loading_matrices = []
        self.uncertainity = []
        self.soft_discrete_latent = np.zeros((self.m, self.G))
        self.mean_within_clusters = []
        self.covariance_within_clusters =[]
        self.B_parameter_within_clusters = []       
        
        if initialization_type == "random":
            for g in range(0, self.G):
                self.mixing_parameter[g] = abs(np.random.randn())
                self.loading_matrices.append((30*np.random.randn()+43) * np.dot(np.eye(self.n), np.ones((self.n, self.loadingSize[g])) ))#size of dxk
                self.uncertainity.append((34*np.random.randn()+43) * np.eye(self.n))
                a = np.random.randn(self.n, self.n)
                self.covariance_within_clusters.append(np.dot(a, a.T))#Creating a random covariance matrix
                a = np.random.randn(self.n).reshape(-1, 1)
                self.mean_within_clusters.append(a)
                self.B_parameter_within_clusters.append(self.computing_B(self.loading_matrices[g], self.uncertainity[g]))
            
            self.mixing_parameter = self.mixing_parameter/np.sum(self.mixing_parameter)#To insure that they sum to 1
            
            for i in range(0, self.m):
                temp = abs(np.random.randn(self.G))
                self.soft_discrete_latent[i, :] = temp/np.sum(temp)
        
        else:
            model = kmeans(self.X_train, self.G)
            centers, clusters  = model.fit(max_iterations=max_iterations)
            print("Finished the initialization by the kmeans")
            for k in range(0, self.G):
                cluster_separated = list(map(lambda i: True if i == k else False, clusters))

                self.mixing_parameter[k] = np.sum(cluster_separated)/self.m
                self.mean_within_clusters.append(centers[k].reshape(-1, 1))
                #THe following is done to create a list for each component
                self.loading_matrices.append((30*np.random.randn()+43) * np.dot(np.eye(self.n), np.ones((self.n, self.loadingSize[k])) ))#size of dxk
                self.uncertainity.append((34*np.random.randn()+43) * np.eye(self.n))#Initial Value
                self.B_parameter_within_clusters.append(self.computing_B(self.loading_matrices[k], self.uncertainity[k]))#Initial Value
                a = np.random.randn(self.n, self.n)
                self.covariance_within_clusters.append(np.dot(a, a.T))#Creating a random covariance matrix
                
                self.soft_discrete_latent[:, k] = np.array(list(cluster_separated))
            
            #for i in range(0, self.m):
                #temp = abs(np.random.randn(self.G))
                #self.soft_discrete_latent[i, :] = temp/np.sum(temp) 
                
            #Calaculating the covariance matrix matrix within each cluster, loading factors and uncertainity matrix within each cluster
            temp = np.zeros((self.n, self.n))
            for g in range(0, self.G):
                self.covariance_within_clusters[g] = self.computing_S(g)
                evalue, evectors = np.linalg.eig(self.covariance_within_clusters[g])
                self.loading_matrices[g] = np.multiply(np.sqrt(evalue[0:self.loadingSize[g]].reshape(1, -1)), evectors[:, 0:self.loadingSize[g]])
                self.uncertainity[g] = abs(np.eye(self.n) *( self.covariance_within_clusters[g] - np.dot( np.sqrt(evalue.reshape(1, -1)) * evectors, (np.sqrt(evalue.reshape(1, -1)) * evectors)).T))
                #print(self.uncertainity[g])
                self.B_parameter_within_clusters[g] = self.computing_B(self.loading_matrices[g], self.uncertainity[g])#Initial Value

            
            self.computing_soft_latent()
        

    def computing_inv_cov_x(self, lambd, psi):
        #Using Woodbury Identity
        inv_psi = np.linalg.inv(psi)
        t1 = np.linalg.inv( np.add(np.eye(lambd.shape[1]), np.dot(lambd.T, np.dot(inv_psi, lambd))) )
        t2 = np.dot(lambd, np.dot(t1, lambd.T))
        t3 = np.dot(inv_psi, np.dot(t2, inv_psi))
        assert(t3.shape == inv_psi.shape)

        return np.subtract(inv_psi, t3)
    
    def computing_B(self, lambd, psi):
        return np.dot(lambd.T, self.computing_inv_cov_x(lambd, psi))
    
    def computing_S(self, g):
        temp = np.zeros((self.n, self.n))
        for i in range(0, self.m):
            temp += self.soft_discrete_latent[i, g] * np.dot(self.X_train[i, :].reshape(-1, 1) - self.mean_within_clusters[g].reshape(-1, 1), (self.X_train[i, :].reshape(-1, 1) - self.mean_within_clusters[g].reshape(-1, 1)).T)
        return temp/np.sum(self.soft_discrete_latent[:, g])
    
    def computing_mean(self, g):
        return np.sum(np.multiply(self.soft_discrete_latent[:, g].reshape(-1, 1), self.X_train), axis=0).reshape(-1, 1)/np.sum(self.soft_discrete_latent[:, g])
    
    def computing_lambda(self, g):
        theta = np.add(np.eye(self.loadingSize[g]), 
        np.subtract(np.dot(self.B_parameter_within_clusters[g], np.dot(self.covariance_within_clusters[g],self.B_parameter_within_clusters[g].T)) , 
        np.dot(self.B_parameter_within_clusters[g], self.loading_matrices[g]) ) )
        return np.dot(self.covariance_within_clusters[g], np.dot(self.B_parameter_within_clusters[g].T, np.linalg.inv(theta) ))

    def computing_psi(self, g):
        temp = np.dot(self.loading_matrices[g], np.dot(self.B_parameter_within_clusters[g], self.covariance_within_clusters[g]))
        return np.subtract(self.covariance_within_clusters[g], temp)
    
    def computing_multivariate(self, g, x):
        #print(self.mean_within_clusters[g].reshape(1, -1).shape)
        #print(x.shape)
        return scipy.stats.multivariate_normal.pdf(x.ravel(), mean=self.mean_within_clusters[g].ravel(), cov=np.dot(self.loading_matrices[g], self.loading_matrices[g].T) + self.uncertainity[g])

    def computing_soft_latent(self):
        temp = np.zeros((self.m, self.G))
        for i in range(0, self.m):
            denominator = 0 
            for k in range(0, self.G):
                denominator = denominator + ( self.computing_multivariate(k, self.X_train[i, :].reshape(-1, 1)) * self.mixing_parameter[k])
            for g in range(0, self.G):
                #print(denominator)
                #1e-50 were added to preven division by zero when computing the log-likelihood
                temp[i, g] = (self.computing_multivariate(g, self.X_train[i, :].reshape(-1, 1)) * self.mixing_parameter[g])/denominator
                #print(temp[i, g])
        self.soft_discrete_latent = temp
    
    def E_step(self, count=2):
        for g in range(0, self.G):
            self.mean_within_clusters[g] = self.computing_mean(g)
            self.mixing_parameter[g] = (1/self.m) * np.sum(self.soft_discrete_latent[:, g])
        
        if count != 0:
            self.computing_soft_latent()
        
        for g in range(0, self.G):
            self.covariance_within_clusters[g] = self.computing_S(g)
            self.B_parameter_within_clusters[g] = self.computing_B(self.loading_matrices[g], self.uncertainity[g])

    def M_step(self):
        for g in range(0, self.G):    
            self.loading_matrices[g] = self.computing_lambda(g)
            self.uncertainity[g] = np.eye(self.n) * self.computing_psi(g)
        
        self.computing_soft_latent()

    def compute_log_likelihood(self):
        temp = 0
        for i in range(0, self.m):
            temp2 = 0
            for g in range(0, self.G):
                temp2 += self.mixing_parameter[g] * self.computing_multivariate(g, self.X_train[i, :])
            #if temp2 <=0:
                #print(temp2)
            #assert(temp2 > 0)
            #print(temp2)
            temp += np.log(temp2)
        
        return temp

    def fit(self, max_iteration, eps=1e-2):
        convergence_test = True
        count = 0
        while( (convergence_test == True) and (count != max_iteration)):
            log_likelihood_t = self.compute_log_likelihood()            
            self.E_step(count)#Update the soft latent values
            temp_psi, temp_lambda = (self.uncertainity.copy(), self.loading_matrices.copy())            
            self.M_step()#Update the parameters of the conditional distribution of x given z and u
            self.computing_soft_latent()#Updating the soft values for cluster assignment
            log_likelihood_t_future = self.compute_log_likelihood()
            print(f"Number of iteration:{count}, max_iteration:{max_iteration}, past:{log_likelihood_t}, future:{log_likelihood_t_future}")
            count = count + 1
            if log_likelihood_t_future != log_likelihood_t_future:#The usual trick nan doesn't equal itself
                self.uncertainity, self.loading_matrices = (temp_psi, temp_lambda)
                print("Something wrong happened in the Maximization step")
                break
            #print(log_likelihood_t_future[0])
            if( (log_likelihood_t_future - log_likelihood_t) < eps and (count > 10)):
                self.uncertainity, self.loading_matrices = (temp_psi, temp_lambda)
                print("We converged to the optimal value for the log-likelihood")
                convergence_test =False #We reached the parameters that maximize the log-likelihood, no adancement in the log-likelihood
            #[print(self.loading_matrices[g].shape) for g in range(0, self.G)]
        
        return self.uncertainity, self.loading_matrices, self.mixing_parameter

    def prediction_dataset(self, X):
        prediciton = []
        for i in range(0, X.shape[0]):
            prediciton.append(self.predict(X[i, :]))
            
        return np.array(prediciton)
    
    def predict(self, x):
        prediction = np.zeros((self.G, 1))
        for g in range(0, self.G):
            denominator = 0 
            for k in range(0, self.G):
                denominator = denominator + ( self.computing_multivariate(k, x.reshape(-1, 1)) * self.mixing_parameter[k])
            prediction[g] = (self.computing_multivariate(g, x.reshape(-1, 1)) * self.mixing_parameter[g])/denominator
        
        return np.argmax(prediction)


In [191]:
numpy.random.seed(120)

#Using IRIS and Wine Dataset
#X, y = sklearn.datasets.load_iris(return_X_y=True)
X, y = sklearn.datasets.load_wine(return_X_y=True)

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=42)
#standard = sklearn.preprocessing.StandardScaler()
#X_train = standard.fit_transform(X_train)
training_data = np.c_[X_train, y_train]#All of the features are continuous, so, no need to use one-hot encoder and we can directly standard normalize the features of the data set

#X_test = standard.transform(X_test)
test_data = np.c_[X_test, y_test]
print(training_data.shape)
print(test_data.shape)
k = len(set(y_train))
y_train#It needs to be labeled from 0 to k
print(k)


(133, 14)
(45, 14)
3


In [192]:
#Randomly Initialized
#Best resulted are obtained by initializing the model by kmeans
#u = [3, 3, 3] #for iris
u = [9, 9, 9] #Lets try projecting it to a lower space
model = Mixtures_Factor_Analyzers_Model(X_train, k, u, "kmeans")
uncertainity, loading_matrices, mixing_parameter = model.fit(1000)

count:0, max_iterations100, past:10, future:0
count:1, max_iterations100, past:10792010.491995996, future:3102217.0489981584
count:2, max_iterations100, past:3102217.0489981584, future:1978365.5976835634
count:3, max_iterations100, past:1978365.5976835634, future:1795178.7694889628
count:4, max_iterations100, past:1795178.7694889628, future:1702081.6684963438
count:5, max_iterations100, past:1702081.6684963438, future:1633691.812665524
count:6, max_iterations100, past:1633691.812665524, future:1616124.000250265
count:7, max_iterations100, past:1616124.000250265, future:1615357.3439364806
Finished the initialization by the kmeans
Number of iteration:0, max_iteration:1000, past:[-2669.05191283], future:[-2300.75165462]
Number of iteration:1, max_iteration:1000, past:[-2300.75165462], future:[-2239.35288695]
Number of iteration:2, max_iteration:1000, past:[-2239.35288695], future:[-2209.44990541]
Number of iteration:3, max_iteration:1000, past:[-2209.44990541], future:[-2192.6340433]
Numb

In [194]:
pred = model.prediction_dataset(X_train)
print("Performance on the training set")
#print(sklearn.metrics.confusion_matrix(y_train, pred))
c = sklearn.metrics.confusion_matrix(y_train, pred)
#c = c[:, list(np.argmax(c, axis=1))]#ordering the cluster to where it shows the highest number of matching with the true labels
c

Performance on the training set


array([[44,  0,  0],
       [ 2, 45,  6],
       [ 0,  6, 30]], dtype=int64)

In [195]:
pred = model.prediction_dataset(X_test)
print("Performance on the test set")
#print(sklearn.metrics.confusion_matrix(y_test, pred))
c = sklearn.metrics.confusion_matrix(y_test, pred)
#c = c[:, list(np.argmax(c, axis=1))]#ordering the cluster to where it shows the highest number of matching with the true labels
c

Performance on the test set


array([[15,  0,  0],
       [ 2, 15,  1],
       [ 0,  1, 11]], dtype=int64)

In [184]:
numpy.random.seed(120)

#Using IRIS  
X, y = sklearn.datasets.load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=42)
k = len(set(y_train))
y_train#It needs to be labeled from 0 to k
print(k)
#Best resulted are obtained by initializing the model by kmeans
u = [2, 2, 2] #Lets try projecting it to a lower space
model = Mixtures_Factor_Analyzers_Model(X_train, k, u, "kmeans")
uncertainity, loading_matrices, mixing_parameter = model.fit(1000)
pred = model.prediction_dataset(X_train)
print("Performance on the training set")
#print(sklearn.metrics.confusion_matrix(y_train, pred))
c = sklearn.metrics.confusion_matrix(y_train, pred)
c = c[:, list(np.argmax(c, axis=1))]#ordering the cluster to where it shows the highest number of matching with the true labels
print(c)
pred = model.prediction_dataset(X_test)
print("Performance on the test set")
#print(sklearn.metrics.confusion_matrix(y_test, pred))
c = sklearn.metrics.confusion_matrix(y_test, pred)
c = c[:, list(np.argmax(c, axis=1))]#ordering the cluster to where it shows the highest number of matching with the true labels
print(c)

3
count:0, max_iterations100, past:10, future:0
count:1, max_iterations100, past:134.96000000000004, future:96.15928353392638
count:2, max_iterations100, past:96.15928353392638, future:61.83372617547311
count:3, max_iterations100, past:61.83372617547311, future:60.34434195402301
count:4, max_iterations100, past:60.34434195402301, future:60.19166954022988
Finished the initialization by the kmeans
Number of iteration:0, max_iteration:1000, past:[-209.17174333], future:[-170.74044787]
Number of iteration:1, max_iteration:1000, past:[-170.74044787], future:[-159.17591952]
Number of iteration:2, max_iteration:1000, past:[-159.17591952], future:[-151.34666362]
Number of iteration:3, max_iteration:1000, past:[-151.34666362], future:[-146.05020933]
Number of iteration:4, max_iteration:1000, past:[-146.05020933], future:[-142.20354696]
Number of iteration:5, max_iteration:1000, past:[-142.20354696], future:[-139.71990125]
Number of iteration:6, max_iteration:1000, past:[-139.71990125], future:[

In [38]:
scipy.stats.multivariate_normal.pdf(np.array([10, 10, 10]), mean=[0, 0, 0], cov=np.eye(3))

4.55572931513341e-67

### References 
* Chapter 2, Chapter 9 and Chapter 12 from Bishop, C. (2006). Pattern Recognition and Machine Learning. Cambridge: Springer.
* Andrew Ng, Lec 13: (https://www.youtube.com/watch?v=LBtuYU-HfUg)
* Andrew Ng, Lec 14: (https://www.youtube.com/watch?v=ey2PE5xi9-A)
* Chapter 3 from McNicholas, P.D. (2016). Mixture Model-Based Classification. Boca Raton: Chapman &
Hall/CRC Press.
